## HYPERPARAMETERS

In [ ]:
PLAYER_COUNT = 4

OPTIMIZER = "Adam"
LOSS = "MeanSquaredError"

LAYER1_NODE = 256
LAYER2_NODE = 256

DATA_SIZE = 1000
EPOCH = 1000

## Initialize board

In [ ]:
from monopoly.board import MonopolyBoard

board_data = MonopolyBoard(PLAYER_COUNT)
board_data.randomize()
print(board_data)

## Check the version and the device to be ran on

In [ ]:
from tensorflow.python.client import device_lib
import tensorflow as tf

print("Tensorflow version:", tf.version.VERSION)
print(device_lib.list_local_devices())

## Initialize summary writer

It is required to use tensorboard.

In [ ]:
import datetime
from tensorboard.plugins.hparams import api as hp

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/" + current_time
summary_writer = tf.summary.create_file_writer(log_dir)

with summary_writer.as_default():
    hp.hparams({
        "player-count": PLAYER_COUNT,
        "optimizer": OPTIMIZER,
        "loss": LOSS,
        "layer1/node": LAYER1_NODE,
        "layer2/node": LAYER2_NODE,
        "data-size": DATA_SIZE,
        "epoch": EPOCH
    })

## Initialize model

In [ ]:
from analyzer.models import MonopolyModel
from analyzer.trainer import MonopolyAITrainer

model = MonopolyModel(LAYER1_NODE, LAYER2_NODE, PLAYER_COUNT)
loss_object = tf.keras.losses.get(LOSS)
optimizer = tf.keras.optimizers.get(OPTIMIZER)

train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()

trainer = MonopolyAITrainer(model, loss_object, optimizer, train_loss, test_loss)

## Train the model

In [ ]:
from analyzer.datageneration import generate_data

trainer.reset()

for epoch in range(EPOCH):
    trainer.train(DATA_SIZE)

    with summary_writer.as_default():
        tf.summary.scalar("loss/train", trainer.train_loss.result(), step=epoch)

    trainer.test(DATA_SIZE)

    with summary_writer.as_default():
        tf.summary.scalar("loss/test", trainer.test_loss.result(), step=epoch)

    if epoch % 10 == 0:
        with summary_writer.as_default():
            tf.summary.histogram("layer1/weights", trainer.model.d1.kernel, step=epoch)
            tf.summary.histogram("layer1/bias", trainer.model.d1.bias, step=epoch)
            tf.summary.histogram("layer2/weights", trainer.model.d2.kernel, step=epoch)
            tf.summary.histogram("layer2/bias", trainer.model.d2.bias, step=epoch)
            tf.summary.histogram("layer3/weights", trainer.model.d3.kernel, step=epoch)
            tf.summary.histogram("layer3/bias", trainer.model.d3.bias, step=epoch)

    trainer.reset()